### Imports

In [36]:
import copy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor

from modAL.models import ActiveLearner, CommitteeRegressor
from modAL.disagreement import max_std_sampling

### Suppresses Warning
import warnings
warnings.filterwarnings('ignore')

seed = 42
random.seed(seed)
np.random.seed(seed)

### Ideas
- Try Gaussian process as regressor. There may be a performance difference using it in Active Learning vs Bayesian Optimization.
- Types of regression to try
    - linear
    - ridge
    - lasso
    - polynomial


### Functions

In [37]:
def get_min_max_combinations(ranges_dict, feature):
    
    # min of all other features, max of this feature
    min_combo = []
    for key in ranges_dict.keys():
        
        if key == feature:
            min_combo.append(min(ranges_dict[key]))
        else:
            min_combo.append(max(ranges_dict[key]))
        

    # max of all other features, min of this feature
    max_combo = []
    for key in ranges_dict.keys():
        
        if key == feature:
            max_combo.append(max(ranges_dict[key]))
        else:
            max_combo.append(min(ranges_dict[key]))
    
    return tuple(min_combo), tuple(max_combo)

In [38]:
def get_index(df, combo):
    
    # split into components
    Mg_glutamate, K_glutamate, Amino_Acid, tRNA, coA, NAD, cAMP, Folinic_Acid, Spermidine, Three_PGA, NTP = combo
    
    # get index of matching row in data
    a = df.index[
              np.isclose(df['Mg-glutamate (mM)'], Mg_glutamate) \
            & np.isclose(df['K-glutamate (mM)'], K_glutamate) \
            & np.isclose(df['Amino Acid (mM)'], Amino_Acid) \
            & np.isclose(df['tRNA (mg/ml)'], tRNA) \
            & np.isclose(df['coA (mM)'],coA)  \
            & np.isclose(df['NAD (mM)'], NAD) \
            & np.isclose(df['cAMP (mM)'], cAMP) \
            & np.isclose(df['Folinic Acid (mM)'], Folinic_Acid) \
            & np.isclose(df['Spermidine (mM)'], Spermidine) \
            & np.isclose(df['3-PGA (mM)'], Three_PGA) \
            & np.isclose(df['NTP (mM)'], NTP)]
    return a.tolist()[0]



In [39]:
def get_initial_22_idx(df, ranges_dict):
    
    initial_idx = []
    
    # for each feature
    for key in ranges_dict.keys():
        min_combo, max_combo = get_min_max_combinations(ranges_dict,key)
        idx = get_index(df, min_combo)
        initial_idx.append(idx)
        idx = get_index(df, max_combo)
        initial_idx.append(idx)
        
    return np.array(initial_idx)
    

# Data Prep

### Load Data

In [40]:
data = pd.read_csv('data/DataPool.csv')
print(data.shape)
data.head(5)
cols = data.columns

(1017, 12)


### Split into features and target

In [41]:
X = data.drop(['Yield'],axis=1)
print(X.shape)
y_pool = data['Yield'].to_numpy()
print(y_pool.shape)

(1017, 11)
(1017,)


### Scale data
So all variables have the same mean and std dev

In [42]:
scaler = StandardScaler()
X_pool = scaler.fit_transform(X)

# Define initial dataset
- 11 data points in which each compound is minimized except one is maximized
- 11 data pionts in which each compound is maximized except one is minimized
- 80 random data points

In [43]:
ranges_dict = {
    'Mg-glutamate (mM)':[0.4,1.2,2,4],
    'K-glutamate (mM)':[8,24,40,80],
    'Amino Acid (mM)': [0.15,0.45,0.75,1.5],
    'tRNA (mg/ml)':[0.02,0.06,0.1,0.2],
    'coA (mM)':[0.026,0.078,0.13,0.26],
    'NAD (mM)':[0.033,0.099,0.165,0.33],
    'cAMP (mM)':[0.075,0.225,0.375,0.75],
    'Folinic Acid (mM)':[0.0068,0.0204,0.034,0.068],
    'Spermidine (mM)':[0.1,0.3,0.5,1],
    '3-PGA (mM)':[3,9,15,30],
    'NTP (mM)':[0.15,0.45,0.75,1.5]
}


In [44]:
idx_first_22 = get_initial_22_idx(data, ranges_dict)

X_training_22 = X_pool[idx_first_22]
y_training_22 = y_pool[idx_first_22]
print(X_training_22.shape)
print(y_training_22.shape)

# remove from pool
X_pool = np.delete(X_pool, idx_first_22, axis=0)
y_pool = np.delete(y_pool, idx_first_22)   
print(X_pool.shape)
print(y_pool.shape)

# chose 80 points from pool at random
idx_random_80 = np.random.choice(range(len(X_pool)), size=80, replace=False)
X_training_80, y_training_80 = X_pool[idx_random_80], y_pool[idx_random_80]
print(X_training_80.shape)
print(y_training_80.shape)

# remove 80 points from pool
X_pool = np.delete(X_pool, idx_random_80, axis=0)
y_pool = np.delete(y_pool, idx_random_80)   
print(X_pool.shape)
print(y_pool.shape) 

# combine training sets together
X_training = np.append(X_training_22, X_training_80, axis=0)
y_training = np.append(y_training_22, y_training_80, axis=0)
print(X_training.shape)
print(y_training.shape)

assert X_training.shape == (102,11)
assert y_training.shape == (102, )

(22, 11)
(22,)
(995, 11)
(995,)
(80, 11)
(80,)
(915, 11)
(915,)
(102, 11)
(102,)


# Define Model

In [45]:
class RfWrapper(RandomForestRegressor):  # superclass
    """
    Wrapper class for RandomForestRegressor which modifies predict() method to include second argument return_std.
    This argument is expected by
    modAL library for active learning regression. Provided by course instructors.
    """

    def predict(self, X, return_std=False):
        if return_std:
            ys = np.array([e.predict(X) for e in self.estimators_])
            return np.mean(ys, axis=0).ravel(), np.std(ys, axis=0).ravel()
        return super().predict(X).ravel()


In [46]:
class LrWrapper(LinearRegression):  #superclass
    """
    Wrapper class for RandomForestRegressor which modifies predict() method to include second argument return_std.
    This argument is expected by
    modAL library for active learning regression.
    """
    
    def predict(self, X, return_std=False):
        if return_std:
            ys = np.array([e.predict(X) for e in self.estimators_])
            return np.mean(ys, axis=0).ravel(), np.std(ys, axis=0).ravel()
        return super().predict(X).ravel()

In [47]:
def get_next_sample(optimizer, X_pool, y_pool):

    # call the query strategy defined in the learner to obtain a new sample
    query_idx, query_sample = optimizer.query(X_pool)

    # modify indexing to interpret as collection of one element with d features
    query_sample_reshaped = query_sample.reshape(1, -1)

    # obtain the query label
    query_label = y_pool[query_idx]

    # modify indexing to interpret as 1D array of one element
    query_label_reshaped = query_label.reshape(1, )

    return query_sample_reshaped, query_label_reshaped, query_idx


In [48]:
def get_next_batch(learner, X_pool, y_pool, batch_size):
    
    n_col = X_pool.shape[1]
    X_batch = np.zeros((batch_size, n_col))
    y_batch = np.zeros((batch_size,))

    
    for i in range(batch_size):
        
        X_sample, y_sample, query_idx = get_next_sample(learner, X_pool, y_pool)
        
        # add to batch
        X_batch[i] = X_sample
        y_batch[i] = y_sample
        
        # remove queried point from pool
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx)     
        
    return X_batch, y_batch, X_pool, y_pool
        


In [49]:
def score_regression_model(optimizer, X_test, y_test):

    y_pred = optimizer.predict(X_test, return_std=False)
    r2 = r2_score(y_test, y_pred)  # y_true, y_pred
    return r2

In [50]:
def build_committee(n_learner, X_training, y_training, seed):
    
    # get bootstrapped initial training set for each learner
    initial_idx = []
    for i in range(n_learner):
        initial_idx.append(np.random.choice(len(X_training), size=len(X_training), replace=True))
        
       
    # initialize learners for Committee
    learner_list = []
    for idx in initial_idx:
        al = ActiveLearner(
            estimator=RfWrapper(n_estimators=20, max_depth=6, random_state=seed),
            X_training=X_training[idx],
            y_training=y_training[idx])
        learner_list.append(al)
        
    # initializing the Committee
    committee = CommitteeRegressor(
        learner_list=learner_list,
        query_strategy=max_std_sampling
    )

    return committee

In [51]:
def run_active_regression(learner, X_pool, y_pool, n_batch, batch_size, initial):
    print('X_pool size, y_pool size', X_pool.shape, y_pool.shape)
#     selected = [initial]
#     data_sets = [(X_pool,y_pool)]
    for b in range(n_batch):
        
        # get next set of points to learn
        X_batch, y_batch, X_pool, y_pool = get_next_batch(learner, X_pool, y_pool, batch_size)
        print('X_pool size, y_pool size', X_pool.shape, y_pool.shape)
        
#         # save data
#         batch_col = np.array([b]*len(X_batch)).reshape(-1,1)
#         y_col = y_batch.reshape(-1,1)
#         result = np.append(X_batch,y_col, axis=1)
#         selected.append(result)
        
#         # save dataset
#         data_sets.append((X_pool,y_pool))
    
        # use new sample to update the model
        learner.teach(X_batch, y_batch)
    
#     return selected,data_sets

In [53]:
%%time

# copy data for this section
X_cp = copy.deepcopy(X_pool)
y_cp = copy.deepcopy(y_pool)
print(X_cp.shape)
print(y_cp.shape)



# define committee
committee = build_committee(25, X_training, y_training, seed)

# save initial training data as first batch
initial = np.append(X_training, y_training.reshape(-1,1),axis=1)

# active regression
n_batch = 10
batch_size = 1
run_active_regression(committee, X_cp, y_cp, n_batch, batch_size, initial)





(915, 11)
(915,)
X_pool size, y_pool size (915, 11) (915,)
X_pool size, y_pool size (914, 11) (914,)
X_pool size, y_pool size (913, 11) (913,)
X_pool size, y_pool size (912, 11) (912,)
X_pool size, y_pool size (911, 11) (911,)
X_pool size, y_pool size (910, 11) (910,)
X_pool size, y_pool size (909, 11) (909,)
X_pool size, y_pool size (908, 11) (908,)
X_pool size, y_pool size (907, 11) (907,)
X_pool size, y_pool size (906, 11) (906,)
X_pool size, y_pool size (905, 11) (905,)
CPU times: user 6.42 s, sys: 89.9 ms, total: 6.51 s
Wall time: 6.54 s
